<a href="https://colab.research.google.com/gist/JakeMalis/5b43ed40dc7e28518f5605dc8fcb35c7/roboflow-lucky-charms-inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this tutorial we will use TensorFlow's Object Detection API and Roboflow's labeling platform to train and evaluate a model to detect marshmallows in lucky charms.

#### ![Roboflow](https://media-exp1.licdn.com/dms/image/C4D1BAQEf8_Y4I1CJwA/company-background_10000/0/1631290065486?e=2159024400&v=beta&t=0RCuQDNh2uELCij-75lsTMJFgCcCzveJh7fOJNaCW3E)

In [ ]:
project_name = "lucky-charms-detection"

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gcloud config set project {project_name}

## Download data

In [ ]:
model = "efficientdet_d4_coco17_tpu-32"

In [ ]:
bucket = 'colab-notebook-4f2d98b4-c663-11ec-a7b0-0242ac1c0002'

In [ ]:
!gsutil cp -r gs://{bucket}/export .
!gsutil cp -r gs://{bucket}/{model} .
!gsutil cp gs://{bucket}/dataset/train/marbits_label_map.pbtxt ./label_map.pbtxt
!gsutil cp gs://{bucket}/model.config .

# Downloads and Installations

## Download and install the TensorFlow Object Detection API

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
%%capture
%cd /content/models/research/
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

# Run model inference

## Configure inference using TensorFlow

In [ ]:
%cd /content

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap('label_map.pbtxt', use_display_name=True)

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
    
  return output_dict

In [ ]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [ ]:
detection_model = tf.saved_model.load('export/saved_model')

## Run detection on uploaded images to root directory

In [ ]:
import pathlib

PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)